# EEG Motor Movement/Imagery Dataset

Published: Sept. 9, 2009. Version: 1.0.0

https://physionet.org/content/eegmmidb/1.0.0/

In [1]:
class Physionet_2009_Dataset():

    EVENT_MAP_DICT = {
        "769": 0,
        "770": 1,
        "771": 2,
        "772": 3,
#        "768": 4,
    }

    UNKNOWN_EVENT_MAP_DICT = {
        "783": 10,
    }

    EOG_CHANNELS = ["EOG-left", "EOG-central", "EOG-right"]

    REJECT_MAGNITUDE = 1e-4

    SUBJECT_INFO_KEYS = [
        "id",
        "smoking",
        "alcohol_abuse",
        "drug_abuse",
        "medication",
        "weight",
        "height",
        "sex",
        "handedness",
        "age",
    ]

    def __init__(self):
        pass

    @classmethod
    def load_as_raw(cls, filepath, load_eog=False, preload=False):
        
        gdf_obj = read_raw_gdf(
            filepath,
            preload=preload,
            eog=cls.EOG_CHANNELS,
            exclude=list() if load_eog else cls.EOG_CHANNELS,
        )
    
        return gdf_obj

    @classmethod
    def load_as_epochs(cls, filepath, tmin=-.3, tmax=.7, reject=None, load_eog=False, has_labels=True):
        if reject is None:
            reject = dict(eeg=cls.REJECT_MAGNITUDE)
        elif reject is False:
            reject = None

        gdf_obj = cls.load_as_raw(filepath, preload=True, load_eog=load_eog)
        events, _ = events_from_annotations(gdf_obj, event_id=cls.EVENT_MAP_DICT if has_labels else cls.UNKNOWN_EVENT_MAP_DICT)
        epochs = Epochs(
            gdf_obj,
            events,
            event_repeated="drop",
            reject_by_annotation=True,
            tmin=tmin,
            tmax=tmax,
            reject=reject,
            baseline=None,
            # proj=False
        )
        return epochs

    @classmethod
    def _parse_info(cls, info_dict):

        parsed_info = {k: v for k, v in info_dict.items() if k in cls.SUBJECT_INFO_KEYS}
        return parsed_info

    @classmethod
    def read_metadata_from_raw(cls, raw):
        return cls._parse_info(
            raw._raw_extras[0]["subject_info"]
        )

    @classmethod
    def read_metadata(cls, filepath):
        gdf_obj = cls.load_as_raw(filepath, preload=False)
        return cls.read_metadata_from_raw(gdf_obj)

    @classmethod
    def load_dataset(cls, filepaths, as_epochs=False, concatenate=False, drop_bad=False, return_metadata=False, **load_kwargs):
        
        objs = list()
        load_fn = cls.load_as_epochs if as_epochs else cls.load_as_raw
        concatenate_fn = concatenate_epochs if as_epochs else concatenate_raws

        objs = list()
        metadata = list()
        for filepath in filepaths:
            obj = load_fn(filepath, **load_kwargs)
            objs.append(obj)

            if return_metadata:
                metadata.append(
                    cls.read_metadata(filepath)
                )
        if concatenate:
            objs = concatenate_fn(objs)
        
        if return_metadata:
            return objs, metadata
            
        return objs


In [41]:
from mne.io import read_raw_edf
from pathlib import Path

path = Path("/home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S003/S003R01.edf")

edf = read_raw_edf(path)

Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S003/S003R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [42]:
edf.get_data().shape

(64, 9760)

In [43]:
events, _ = events_from_annotations(edf)
events, _

Used Annotations descriptions: ['T0']


(array([[0, 0, 1]]), {'T0': 1})

In [44]:
from mne import concatenate_raws

sample_paths = path.parent.glob("*.edf")
edf_list = list()
for edf_path in sample_paths:
    edf_list.append(
        read_raw_edf(edf_path)
    )
c_edf = concatenate_raws(edf_list)

Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S003/S003R07.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S003/S003R05.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S003/S003R02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S003/S003R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/file

In [45]:
c_edf.get_data().shape

(64, 259520)

In [46]:
events, _ = events_from_annotations(c_edf)
events, _

Used Annotations descriptions: ['T0', 'T1', 'T2']


(array([[     0,      0,      1],
        [   672,      0,      2],
        [  1328,      0,      1],
        ...,
        [257456,      0,      3],
        [258112,      0,      1],
        [258784,      0,      2]]),
 {'T0': 1, 'T1': 2, 'T2': 3})

In [47]:
epochs = Epochs(
    c_edf,
    events,
    event_repeated="drop",
    reject_by_annotation=True,
    tmin=0,
    tmax=1,
    reject=None,
    baseline=None,
    # proj=False
)

Not setting metadata
362 matching events found
No baseline correction applied
0 projection items activated


In [48]:
epochs.get_data().shape

Loading data for 362 events and 161 original time points ...
0 bad epochs dropped


(362, 64, 161)

In [69]:
edf = read_raw_edf("/home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S002/S002R06.edf")
edf.get_data().shape

Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S002/S002R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


(64, 19680)

In [86]:
path = Path("/home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/")
shape_dict = dict()
for uid in range(1, 110):
    sub_uid = "S" + str(uid).rjust(3, "0")
    sub_path = path / sub_uid
    
    edf_filepaths = sub_path.glob("*.edf")
    
    c_edf = concatenate_raws(
        [
            # Some sujects, as sub 88, has data sampled at 128Hz
            read_raw_edf(edf_path).resample(160)
            for edf_path
            in edf_filepaths
        ]
    )
    events, _ = events_from_annotations(c_edf)

    epochs = Epochs(
        c_edf,
        events,
        event_repeated="drop",
        reject_by_annotation=True,
        tmin=0.01,
        tmax=.9,
        reject=None,
        baseline=None,
        # proj=False
    )
    shape_dict[sub_uid] = epochs.get_data().shape

Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S001/S001R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S001/S001R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S001/S001R13.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S001/S001R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/file

Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S003/S003R12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S003/S003R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S003/S003R08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S003/S003R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-mov

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Not setting metadata
362 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 362 events and 143 original time points ...
0 bad epochs dropped
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S006/S006R13.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S006/S006R02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S006/S006R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure..

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S008/S008R09.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S008/S008R05.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S008/S008R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S008/S008R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /h

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S010/S010R11.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Not setting metadata
362 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 362 events and 143 original time points ...
0 bad epochs dropped
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S011/S011R08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S011/S011R02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure..

...

/tmp/ipykernel_234220/3559815916.py:12: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  read_raw_edf(edf_path).resample(160)


Used Annotations descriptions: ['T0', 'T1', 'T2']
Not setting metadata
290 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 290 events and 143 original time points ...
0 bad epochs dropped
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S101/S101R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S101/S101R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S101/S101R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S103/S103R09.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S103/S103R12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S103/S103R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S103/S103R02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /h

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S105/S105R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Not setting metadata
362 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 362 events and 143 original time points ...
0 bad epochs dropped
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S106/S106R08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S106/S106R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure..

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S108/S108R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S108/S108R07.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S108/S108R02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S108/S108R09.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /h

In [87]:
shape_dict

{'S001': (362, 64, 143),
 'S002': (362, 64, 143),
 'S003': (362, 64, 143),
 'S004': (362, 64, 143),
 'S005': (362, 64, 143),
 'S006': (362, 64, 143),
 'S007': (362, 64, 143),
 'S008': (362, 64, 143),
 'S009': (362, 64, 143),
 'S010': (362, 64, 143),
 'S011': (362, 64, 143),
 'S012': (362, 64, 143),
 'S013': (362, 64, 143),
 'S014': (362, 64, 143),
 'S015': (362, 64, 143),
 'S016': (362, 64, 143),
 'S017': (362, 64, 143),
 'S018': (362, 64, 143),
 'S019': (362, 64, 143),
 'S020': (362, 64, 143),
 'S021': (362, 64, 143),
 'S022': (362, 64, 143),
 'S023': (362, 64, 143),
 'S024': (362, 64, 143),
 'S025': (362, 64, 143),
 'S026': (362, 64, 143),
 'S027': (362, 64, 143),
 'S028': (362, 64, 143),
 'S029': (362, 64, 143),
 'S030': (362, 64, 143),
 'S031': (362, 64, 143),
 'S032': (362, 64, 143),
 'S033': (362, 64, 143),
 'S034': (362, 64, 143),
 'S035': (362, 64, 143),
 'S036': (362, 64, 143),
 'S037': (362, 64, 143),
 'S038': (362, 64, 143),
 'S039': (362, 64, 143),
 'S040': (362, 64, 143),


In [85]:
sub_uid = "S" + str(90).rjust(3, "0")
sub_path = path / sub_uid

    
edf_filepaths = sub_path.glob("*.edf")

[
    (read_raw_edf(edf_path, verbose=0).get_data().shape, read_raw_edf(edf_path, verbose=0).info["sfreq"])
    for edf_path
    in edf_filepaths
]

[((64, 19680), 160.0),
 ((64, 19680), 160.0),
 ((64, 9760), 160.0),
 ((64, 19680), 160.0),
 ((64, 19680), 160.0),
 ((64, 19680), 160.0),
 ((64, 19680), 160.0),
 ((64, 19680), 160.0),
 ((64, 19680), 160.0),
 ((64, 9760), 160.0),
 ((64, 19680), 160.0),
 ((64, 19680), 160.0),
 ((64, 19680), 160.0),
 ((64, 19680), 160.0)]

In [80]:
9760/160

61.0

In [63]:
    edf_filepaths = sub_path.glob("*.edf")

[
    read_raw_edf(edf_path).get_data().shape
    for edf_path
    in edf_filepaths
]


Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S002/S002R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S002/S002R11.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S002/S002R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/files/S002/S002R02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/paulo/Documents/datasets/Physionet/eeg-motor-movementimagery-dataset-1.0.0/file

[(64, 9760),
 (64, 19680),
 (64, 19680),
 (64, 9760),
 (64, 19680),
 (64, 19680),
 (64, 19680),
 (64, 19680),
 (64, 19680),
 (64, 19680),
 (64, 19680),
 (64, 19680),
 (64, 19680),
 (64, 19680)]

In [64]:
edf

<RawEDF | S003R01.edf, 64 x 9760 (61.0 s), ~64 kB, data not loaded>

In [17]:
from mne.io import read_raw_gdf
from mne import Epochs, events_from_annotations, concatenate_raws, concatenate_epochs


PRELOAD = False

class BCI_IV_Comp_Dataset():

    EVENT_MAP_DICT = {
        "769": 0,
        "770": 1,
        "771": 2,
        "772": 3,
#        "768": 4,
    }

    UNKNOWN_EVENT_MAP_DICT = {
        "783": 10,
    }

    EOG_CHANNELS = ["EOG-left", "EOG-central", "EOG-right"]

    REJECT_MAGNITUDE = 1e-4

    SUBJECT_INFO_KEYS = [
        "id",
        "smoking",
        "alcohol_abuse",
        "drug_abuse",
        "medication",
        "weight",
        "height",
        "sex",
        "handedness",
        "age",
    ]

    def __init__(self):
        pass

    @classmethod
    def load_as_raw(cls, filepath, load_eog=False, preload=False):
        
        gdf_obj = read_raw_gdf(
            filepath,
            preload=preload,
            eog=cls.EOG_CHANNELS,
            exclude=list() if load_eog else cls.EOG_CHANNELS,
        )
    
        return gdf_obj

    @classmethod
    def load_as_epochs(cls, filepath, tmin=-.3, tmax=.7, reject=None, load_eog=False, has_labels=True):
        if reject is None:
            reject = dict(eeg=cls.REJECT_MAGNITUDE)
        elif reject is False:
            reject = None

        gdf_obj = cls.load_as_raw(filepath, preload=True, load_eog=load_eog)
        events, _ = events_from_annotations(gdf_obj, event_id=cls.EVENT_MAP_DICT if has_labels else cls.UNKNOWN_EVENT_MAP_DICT)
        return events, _
        epochs = Epochs(
            gdf_obj,
            events,
            event_repeated="drop",
            reject_by_annotation=True,
            tmin=tmin,
            tmax=tmax,
            reject=reject,
            baseline=None,
            # proj=False
        )
        return epochs

    @classmethod
    def _parse_info(cls, info_dict):

        parsed_info = {k: v for k, v in info_dict.items() if k in cls.SUBJECT_INFO_KEYS}
        return parsed_info

    @classmethod
    def read_metadata_from_raw(cls, raw):
        return cls._parse_info(
            raw._raw_extras[0]["subject_info"]
        )

    @classmethod
    def read_metadata(cls, filepath):
        gdf_obj = cls.load_as_raw(filepath, preload=False)
        return cls.read_metadata_from_raw(gdf_obj)

    @classmethod
    def load_dataset(cls, filepaths, as_epochs=False, concatenate=False, drop_bad=False, return_metadata=False, **load_kwargs):
        
        objs = list()
        load_fn = cls.load_as_epochs if as_epochs else cls.load_as_raw
        concatenate_fn = concatenate_epochs if as_epochs else concatenate_raws

        objs = list()
        metadata = list()
        for filepath in filepaths:
            obj = load_fn(filepath, **load_kwargs)
            objs.append(obj)

            if return_metadata:
                metadata.append(
                    cls.read_metadata(filepath)
                )
        if concatenate:
            objs = concatenate_fn(objs)
        
        if return_metadata:
            return objs, metadata
            
        return objs

BCI_IV_Comp_Dataset.load_as_epochs("/home/paulo/Documents/datasets/BCI_Comp_IV_2a/gdf/A01T.gdf")

Extracting EDF parameters from /home/paulo/Documents/datasets/BCI_Comp_IV_2a/gdf/A01T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG
Creating raw.info structure...
Reading 0 ... 672527  =      0.000 ...  2690.108 secs...


/home/paulo/anaconda3/envs/bci_update/lib/python3.9/site-packages/mne/io/edf/edf.py:1123: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
/home/paulo/anaconda3/envs/bci_update/lib/python3.9/contextlib.py:124: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Used Annotations descriptions: ['769', '770', '771', '772']


(array([[ 92368,      0,      3],
        [ 94371,      0,      2],
        [ 96289,      0,      1],
        [ 98241,      0,      0],
        [100249,      0,      0],
        [102360,      0,      1],
        [104277,      0,      2],
        [106327,      0,      3],
        [108258,      0,      1],
        [110256,      0,      2],
        [112162,      0,      0],
        [114057,      0,      0],
        [116029,      0,      0],
        [117973,      0,      3],
        [119940,      0,      1],
        [122060,      0,      1],
        [124068,      0,      0],
        [126134,      0,      0],
        [128170,      0,      2],
        [130236,      0,      0],
        [132306,      0,      1],
        [134386,      0,      3],
        [136298,      0,      3],
        [138329,      0,      2],
        [140282,      0,      0],
        [142243,      0,      3],
        [144347,      0,      3],
        [146351,      0,      1],
        [148326,      0,      3],
        [15035

In [89]:
from scipy.io import loadmat

path = Path("/home/paulo/Documents/datasets/OpenBMI/sess01_subj01_EEG_MI.mat")

mat = loadmat(path)

In [6]:
import mne
from mne import events_from_annotations
from mne import Epochs

def load_as_epochs(filepath, tmin=-2, tmax=6, reject=None):
    edf_obj = mne.io.read_raw_edf(filepath)
    events, _ = events_from_annotations(edf_obj)
    epochs = Epochs(
        edf_obj,
        events,
        event_repeated="drop",
        reject_by_annotation=True,
        tmin=tmin,
        tmax=tmax,
        reject=reject,
        baseline=None,
        # proj=False
    )
    return epochs
epochs = load_as_epochs("/home/paulo/Documents/datasets/OpenBMI/edf/session1/14_train.edf")

Extracting EDF parameters from /home/paulo/Documents/datasets/OpenBMI/edf/session1/14_train.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['0', '1']
Not setting metadata
100 matching events found
No baseline correction applied
0 projection items activated


/tmp/ipykernel_267700/2237221524.py:6: RuntimeWarning: Highpass cutoff frequency 40.0 is greater than lowpass cutoff frequency 0.5, setting values to 0 and Nyquist.
  edf_obj = mne.io.read_raw_edf(filepath)


In [ ]:
epochs.plot_image()

Loading data for 100 events and 8001 original time points ...
0 bad epochs dropped
Not setting metadata
100 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
